# Building a Spam Filter with Naive Bayes Multinomial Algorithm


In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our goal is to write a program that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on this page, where you can also find some of the papers authored by Tiago A. Almeida and José María Gómez Hidalgo.

## Exploring the Dataset

We'll now start by reading in the dataset.

In [1]:
import pandas as pd
dataset = pd.read_csv('SMSSpamCollection',sep='\t', header =None, names = ['Label', 'SMS'])
dataset.shape

(5572, 2)

In [2]:
dataset.head(5)

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
dataset.describe()

,Label,SMS
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [4]:
dataset['Label'].value_counts(normalize = True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

The training set has 87% Ham and 13% Spam

## Training and Test Set

We're now going to split our dataset into a training and a test set, where the training set accounts for 80% of the data, and the test set for the remaining 20%. We'll then analyze the percentage of spam and ham messages in the training and test sets. We expect the percentages to be close to what we have in the full dataset, where about 87% of the messages are ham, and the remaining 13% are spam.

In [5]:
# randomize the data set

random_dataset = dataset.sample(frac = 1, random_state = 1)
random_dataset.reset_index(inplace = True)
training =  random_dataset.loc[:4457,]
test = random_dataset.loc[4457:,]
test.shape

(1115, 3)

In [6]:
training['Label'].value_counts(normalize = True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [7]:
test['Label'].value_counts(normalize = True)

ham     0.868161
spam    0.131839
Name: Label, dtype: float64

The results look good! We'll now move on to cleaning the dataset.

## Data Cleaning

To calculate all the probabilities required by the algorithm, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

Essentially, we want to bring data to this format:


![table](https://camo.githubusercontent.com/27a4a0a699bd8f0713d73347abe2929c267a03d5/68747470733a2f2f64712d636f6e74656e742e73332e616d617a6f6e6177732e636f6d2f3433332f637067705f646174617365745f332e706e67)


### Letter Case and Punctuation

We'll begin with removing all the punctuation and bringing every letter to lower case.

In [8]:
# before cleaning
training.head(5)

,index,Label,SMS
0,1078,ham,"Yep, by the pretty sculpture"
1,4028,ham,"Yes, princess. Are you going to make me moan?"
2,958,ham,Welp apparently he retired
3,4642,ham,Havent.
4,4674,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [9]:
# after cleaning
training = training.copy() # getting rid of SettingWithCopyWarning 
training['SMS']= training['SMS'].str.replace('\W',' ')
training['SMS'] = training['SMS'].str.lower()
training.head(5)

,index,Label,SMS
0,1078,ham,yep by the pretty sculpture
1,4028,ham,yes princess are you going to make me moan
2,958,ham,welp apparently he retired
3,4642,ham,havent
4,4674,ham,i forgot 2 ask ü all smth there s a card on ...


### Creating the Vocabulary

Let's now move to creating the vocabulary, which in this context means a list with all the unique words in our training set.

In [10]:
training['SMS'] = training['SMS'].str.split()
training.head(5)

,index,Label,SMS
0,1078,ham,"[yep, by, the, pretty, sculpture]"
1,4028,ham,"[yes, princess, are, you, going, to, make, me,..."
2,958,ham,"[welp, apparently, he, retired]"
3,4642,ham,[havent]
4,4674,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


In [11]:
vocabulary = []
for sms in training['SMS']:
    for word in sms:
        vocabulary.append(word)
vocabulary = set(vocabulary)
vocabulary = list(vocabulary)
vocabulary[:10]

['pride',
 'meeting',
 'listening2the',
 'box334sk38ch',
 'rightio',
 'handset',
 'responding',
 'nauseous',
 'geoenvironmental',
 'fated']

In [12]:
len(vocabulary)

7783

### The Final Training Set

We're now going to use the vocabulary we just created to make the data transformation we want.

In [13]:
word_counts_per_sms = {unique_word: [0] * len(training['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        
word_counts_per_sms = pd.DataFrame(word_counts_per_sms)
training_set = pd.concat([training, word_counts_per_sms], axis = 1)

In [14]:
training_set.shape

(4458, 7786)

In [15]:
training_set.loc[0,'yep'] # spot checking 'yep' count of 1

1

## Calculating Constants First

We're now done with cleaning the training set, and we can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

* $P(Spam)$ and $P(Ham)$
* $NSpam$, $NHam$, $NVocabulary$

We'll also use Laplace smoothing and set $\alpha = 1$.

In [16]:
# calculate P(Spam) & P(Ham)

spam_or_ham = training_set['Label'].value_counts(normalize = True)
p_ham = spam_or_ham[0]
p_spam = spam_or_ham[1]

In [17]:
# Calculate Nspam, Nham, Nvocabulary

n_spam = training_set[training_set['Label'] == 'spam']
n_spam = n_spam.iloc[:,3:].values.sum()
print(n_spam)

n_ham = training_set[training_set['Label'] == 'ham']
n_ham = n_ham.iloc[:,3:].values.sum()
print(n_ham)

n_vocabulary = len(vocabulary)
print(n_vocabulary)

15190
57237
7783


In [18]:
# comparing with solution notebook result
spam_messages = training_set[training_set['Label'] == 'spam']
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam_sol = n_words_per_spam_message.sum()
print(n_spam_sol)

ham_messages = training_set[training_set['Label'] == 'ham']
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham_sol = n_words_per_ham_message.sum()
print(n_ham_sol)

15190
57237


In [19]:
# Laplace smoothing: Initiate a variable named alpha with a value of 1.
alpha = 1

## Calculating Parameters

Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:
\begin{equation}
P(\text{"secret"}|Spam) = \frac{N_{"secret"|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [20]:
# initialize dictionaries

parameters_spam = {word: 0 for word in vocabulary}
parameters_ham = {word: 0 for word in vocabulary}

# Iterate over the vocabulary, and, for each word, calculate P(wi|Spam) and P(wi|Ham)

for word in vocabulary:
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha * n_vocabulary)
    parameters_ham[word] = p_word_given_ham
    
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha * n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    

## Classifying A New Message

Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

* Takes in as input a new message (w1, w2, ..., wn).
* Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn).
* Compares the values of $P(Spam|w1, w2, ..., wn)$ and $P(Ham|w1, w2, ..., wn)$, and:
    * If $P(Ham|w1, w2, ..., wn)$ > $P(Spam|w1, w2, ..., wn)$, then the message is classified as `ham`.
    * If $P(Ham|w1, w2, ..., wn)$ < $P(Spam|w1, w2, ..., wn)$, then the message is classified as `spam`.
    * If $P(Ham|w1, w2, ..., wn)$ = $P(Spam|w1, w2, ..., wn)$, then the algorithm may request human help.

In [21]:
import re

def classify(message:str) -> str:
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()


    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')
        
message_1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
message_2 = "Sounds good, Tom, then see u there"

In [22]:
classify(message_1)

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [23]:
classify(message_2)

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


In [24]:
message_3 = "Loads of cash hello juicy fruitey cash money money"
classify(message_3)

P(Spam|message): 1.0328782641954482e-28
P(Ham|message): 3.061890908412371e-28
Label: Ham




## Measuring the Spam Filter's Accuracy

The first two results above look promising, but the third one is not. Let's see how well the filter does on our test set, which has 1,114 messages.

We'll start by writing a function that returns classification labels instead of printing them.

In [25]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set.

In [26]:
test = test.copy()
test.reset_index(drop=True, inplace = True)
test['predicted'] = test['SMS'].apply(classify_test_set)
test.head()

,index,Label,SMS,predicted
0,3482,ham,Wherre's my boytoy ? :-(,ham
1,2131,ham,Later i guess. I needa do mcat study too.,ham
2,3418,ham,But i haf enuff space got like 4 mb...,ham
3,3424,spam,Had your mobile 10 mths? Update to latest Oran...,spam
4,1538,ham,All sounds good. Fingers . Makes it difficult ...,ham


Now, we'll write a function to measure the accuracy of our spam filter to find out how well our spam filter does.

In [27]:
correct = 0
total = test.shape[0]

for index, row in test.iterrows():
    if row['Label'] == row['predicted']:
        correct += 1

accuracy = correct / total

print('The spam filter correctly predicted {} messages out of {}. That is an accuracy score of {:.2f} %'.format(correct, total, accuracy * 100))

The spam filter correctly predicted 1101 messages out of 1115. That is an accuracy score of 98.74 %


In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set, which is an excellent result. We initially aimed for an accuracy of over 80%, but we managed to do way better than that.